In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from keras import backend as K
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

import time

In [6]:
label_encoder = LabelEncoder()
standard_scaler = StandardScaler()

def fit_transform_one_hot(arr):
    output = np.zeros([arr.shape[0]])
    for i in range(len(output)):
        output[i] = np.argmax(arr[i])
    return output

label_encoder.fit_transform_one_hot = fit_transform_one_hot

In [7]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

4422102/4422102 [==============================] - 2s 1us/step


In [10]:
X_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [90]:
print(np.where(y == 'NoCognitiveImpairment')[0].shape[0])
print(np.where(y == 'MildCognitiveImpairment')[0].shape[0])
print(np.where(y == 'AD')[0].shape[0])



115
73
104


In [11]:
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

In [19]:
# n_features = X_train.shape[1]
n_classes = 10  # y_train.shape[1]

# 7, 67, 72

lr_l1 = LogisticRegression(penalty="l2", multi_class="ovr", solver="saga", max_iter=500)
svm = SVC()
rf = RandomForestClassifier(n_estimators=500)

dnn = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(n_classes)
])
dnn.compile(optimizer='adam', 
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])


classical_models = (lr_l1, svm, rf)

In [81]:
y_train_sparse = label_encoder.fit_transform(y_train.ravel())
y_test_sparse = label_encoder.transform(y_test.ravel())


for model in classical_models:
    model.fit(X_train_normalized, y_train_sparse)
    y_pred = model.predict(X_test_normalized).ravel()
    
    cnf = confusion_matrix(y_test_sparse, y_pred)
    acc = accuracy_score(y_test_sparse, y_pred)
    # f1 = f1_score(y_test_sparse, y_pred)

    print(f"acc: {acc}")  # , f1: {f1}")
    print(cnf)
    print("********************")

/home/danielzhang/Projects/alzheimers_diagnosis_sinai/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/danielzhang/Projects/alzheimers_diagnosis_sinai/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/danielzhang/Projects/alzheimers_diagnosis_sinai/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


acc: 0.3333333333333333
[[4 6 6]
 [0 3 5]
 [2 1 3]]
********************
acc: 0.2
[[ 0  0 16]
 [ 0  0  8]
 [ 0  0  6]]
********************
acc: 0.3333333333333333
[[ 4  0 12]
 [ 1  0  7]
 [ 0  0  6]]
********************


In [20]:
dnn.fit(X_train, y_train, epochs=5)
y_pred_train = dnn.predict(X_train)
y_pred_test = dnn.predict(X_test)


y_pred_train_sparse = label_encoder.fit_transform_one_hot(y_pred_train)
y_pred_test_sparse = label_encoder.fit_transform_one_hot(y_pred_test)


cnf_train = confusion_matrix(y_pred_train_sparse, y_train)
acc_train = accuracy_score(y_pred_train_sparse, y_train)

print(f"acc: {acc_train}")
print(cnf_train)
print("********************")

cnf_test = confusion_matrix(y_pred_test_sparse, y_test)
acc_test = accuracy_score(y_pred_test_sparse, y_test)

print(f"acc: {acc_test}")
print(cnf_test)
print("********************")

Epoch 1/5
1875/1875 [==============================] - 1s 687us/step - loss: 3.5651 - accuracy: 0.6921
Epoch 2/5
1875/1875 [==============================] - 1s 690us/step - loss: 0.7272 - accuracy: 0.7341
Epoch 3/5
1875/1875 [==============================] - 1s 692us/step - loss: 0.6249 - accuracy: 0.7684
Epoch 4/5
1875/1875 [==============================] - 1s 696us/step - loss: 0.5534 - accuracy: 0.8051
Epoch 5/5
313/313 [==============================] - 0s 459us/step
acc: 0.8253833333333334
[[4962   26   25  157    3    1 1043    0   10    0]
 [   2 5687    9   19    4    0    3    0    0    0]
 [ 351   78 5130  192 1050   14 1447    3   80   18]
 [ 403  195   88 5456  553    4  268    0   38    2]
 [   3    0  289   74 3479    0  411    0    4    0]
 [   0    0    1    0    3 5530    2  153    5  185]
 [ 212   12  427   88  834    4 2370    2   62    4]
 [   0    0    0    0    0  199    0 5716   14  373]
 [  67    2   31   14   74   78  456   19 5787   12]
 [   0    0    0    

In [15]:
y_pred_train

array([[ 0.00958006,  0.00844916, -0.00099614, ..., -0.01144092,
         0.00686088, -0.00235485],
       [ 0.00958006,  0.00844916, -0.00099614, ..., -0.01144092,
         0.00686088, -0.00235485],
       [ 0.00958006,  0.00844916, -0.00099614, ..., -0.01144092,
         0.00686088, -0.00235485],
       ...,
       [ 0.00958006,  0.00844916, -0.00099614, ..., -0.01144092,
         0.00686088, -0.00235485],
       [ 0.00958006,  0.00844916, -0.00099614, ..., -0.01144092,
         0.00686088, -0.00235485],
       [ 0.00958006,  0.00844916, -0.00099614, ..., -0.01144092,
         0.00686088, -0.00235485]], dtype=float32)

In [13]:
y_train_one_hot = to_categorical(y_train_sparse)
y_test_one_hot = to_categorical(y_test_sparse)
# print(y_train_one_hot)

dnn.fit(X_train, y_train_one_hot, epochs=5)
y_pred_train = dnn.predict(X_train)
y_pred = dnn.predict(X_test)

y_pred_sparse_train = label_encoder.fit_transform_one_hot(y_pred_train)
y_pred_sparse = label_encoder.fit_transform_one_hot(y_pred)

cnf_train = confusion_matrix(y_train_sparse, y_pred_sparse_train)
acc_train = accuracy_score(y_train_sparse, y_pred_sparse_train)

print(f"acc: {acc_train}")
print(cnf_train)
print("********************")

cnf_test = confusion_matrix(y_test_sparse, y_pred_sparse)
acc_test = accuracy_score(y_test_sparse, y_pred_sparse)

print(f"acc: {acc_test}")
print(cnf_test)
print("********************")

NameError: name 'y_train_sparse' is not defined

In [110]:
np.where(y_test_sparse == 2)[0].shape[0]

6

In [ ]:
def encoder(input_encoder):
	inputs = keras.Input(shape=input_encoder, name='input_layer')

	# Block-1
	x = layers.Dense(100, name='conv_1')(inputs)
	x = layers.LeakyReLU(name='lrelu_1')(x)

	# Block-2
	x = layers.Dense(75, name='conv_2')(x)
	x = layers.LeakyReLU(name='lrelu_2')(x)

	# Block-3
	x = layers.Dense(75, name='conv_3')(x)
	x = layers.LeakyReLU(name='lrelu_3')(x)


	# Block-4
	x = layers.Dense(50, name='conv_4')(x)
	x = layers.LeakyReLU(name='lrelu_4')(x)

	# Final Block
	flatten = x
	mean = layers.Dense(2, name='mean')(flatten)
	log_var = layers.Dense(2, name='log_var')(flatten)
	model = tf.keras.Model(inputs, (mean, log_var), name="Encoder")

	return model


def sampling(input_1,input_2):
	mean = keras.Input(shape=input_1, name='input_layer1')
	log_var = keras.Input(shape=input_2, name='input_layer2')
	out = layers.Lambda(sampling_reparameterization_model, name='encoder_output')([mean, log_var])
	enc_2 = tf.keras.Model([mean,log_var], out,  name="Encoder_2")

	return enc_2


def sampling_reparameterization_model(distribution_params):
    mean, log_var = distribution_params
    epsilon = K.random_normal(shape=K.shape(mean), mean=0., stddev=1.)
    z = mean + K.exp(log_var / 2) * epsilon

    return z


# replace the decoder with the supervised learning method
def decoder(input_decoder):
		
	inputs = keras.Input(shape=input_decoder, name='input_layer')
	x = layers.Dense(50, name='dense_1')(inputs)
	
	# Block-1
	x = layers.Dense(75,name='conv_transpose_1')(x)
	x = layers.BatchNormalization(name='bn_1')(x)
	x = layers.LeakyReLU(name='lrelu_1')(x)
	
	# Block-2
	x = layers.Dense(75, name='conv_transpose_2')(x)
	x = layers.BatchNormalization(name='bn_2')(x)
	x = layers.LeakyReLU(name='lrelu_2')(x)		

	# INSERT NORMAL SUPERVISED LEARNING HERE?
	# Block-4
	outputs = layers.Dense(100, name='conv_transpose_4')(x)
	model = tf.keras.Model(inputs, outputs, name="Decoder")
	return model	

optimizer = tf.keras.optimizers.Adam(lr = 0.0005)

def mse_loss(y_true, y_pred):
	r_loss = K.mean(K.square(y_true - y_pred), axis = [1,2,3])
	return 1000 * r_loss
	

def kl_loss(mean, log_var):
	kl_loss =  -0.5 * K.sum(1 + log_var - K.square(mean) - K.exp(log_var), axis = 1)
	return kl_loss
	

def vae_loss(y_true, y_pred, mean, log_var):
	r_loss = mse_loss(y_true, y_pred)
	kl_loss = kl_loss(mean, log_var)
	return  r_loss + kl_loss

@tf.function
def train_step(images):
	
	with tf.GradientTape() as enc, tf.GradientTape() as dec:
		
		mean, log_var = enc(images, training=True)
		latent = sampling([mean, log_var])
		generated_images = dec(latent, training=True)
		loss = vae_loss(images, generated_images, mean, log_var)
	
			
	gradients_of_enc = encoder.gradient(loss, enc.trainable_variables)
	gradients_of_dec = decoder.gradient(loss, dec.trainable_variables)
		
		
	optimizer.apply_gradients(zip(gradients_of_enc, enc.trainable_variables))
	optimizer.apply_gradients(zip(gradients_of_dec, dec.trainable_variables))
	return loss


def train(dataset, epochs):
	for epoch in range(epochs):
		start = time.time()
		for image_batch in dataset:
			train_step(image_batch)
	
	print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))



/home/danielzhang/Projects/alzheimers_diagnosis_sinai/venv/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
